<mark>Refined-Silver → Cleaned, structured data (business logic applied)</mark>

In [ ]:
import json

# Access parameter directly from Raw-Bronze notebook / (Toggle parameter cell)
bronze_output = ""

In [ ]:
# Parse the JSON string
output_data = json.loads(bronze_output)

# Access individual variables
start_date = output_data.get("start_date")
silver_adls = output_data.get("silver_adls")
bronze_adls = output_data.get("bronze_adls")

print(f"Start Date: {start_date}")
print(f"Silver ADLS: {silver_adls}")
print(f"Bronze ADLS: {bronze_adls}")

In [20]:
# For manually testing the notebook / remove before running in Data Factory Pipeline 

#from datetime import date, timedelta

#start_date = date.today() - timedelta(1)

#bronze_adls = "abfss://raw-bronze@synapsetest1298.dfs.core.windows.net/"
#silver_adls = "abfss://refined-silver@synapsetest1298.dfs.core.windows.net/"



StatementMeta(Synapsedemo, 1, 2, Finished, Available, Finished)

In [21]:
# import libraries
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

StatementMeta(Synapsedemo, 1, 3, Finished, Available, Finished)

In [23]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

# print df and head
print(df)
print(df.head())

StatementMeta(Synapsedemo, 1, 5, Finished, Available, Finished)

DataFrame[geometry: struct<coordinates:array<double>,type:string>, id: string, properties: struct<code:string,detail:string,dmin:double,gap:double,ids:string,mag:double,magType:string,net:string,nst:bigint,place:string,rms:double,sig:bigint,sources:string,status:string,time:bigint,title:string,tsunami:bigint,type:string,types:string,updated:bigint,url:string>, type: string]
Row(geometry=Row(coordinates=[-122.817337036133, 38.8286666870117, 1.79999995231628], type='Point'), id='nc75237097', properties=Row(code='75237097', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75237097&format=geojson', dmin=0.007739, gap=82.0, ids=',nc75237097,', mag=0.9, magType='md', net='nc', nst=16, place='8 km NNW of The Geysers, CA', rms=0.02, sig=12, sources=',nc,', status='automatic', time=1757721597550, title='M 0.9 - 8 km NNW of The Geysers, CA', tsunami=0, type='earthquake', types=',nearby-cities,origin,phase-data,scitech-link,', updated=1757723237978, url='https://earthquake.usgs.

In [24]:
# Reshape the data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

# print df and head
print(df)
print(df.head())

StatementMeta(Synapsedemo, 1, 6, Finished, Available, Finished)

DataFrame[id: string, longitude: double, latitude: double, elevation: double, title: string, place_description: string, sig: bigint, mag: double, magType: string, time: bigint, updated: bigint]
Row(id='nc75237097', longitude=-122.817337036133, latitude=38.8286666870117, elevation=1.79999995231628, title='M 0.9 - 8 km NNW of The Geysers, CA', place_description='8 km NNW of The Geysers, CA', sig=12, mag=0.9, magType='md', time=1757721597550, updated=1757723237978)


In [25]:
# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)


# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

df.head()

StatementMeta(Synapsedemo, 1, 7, Finished, Available, Finished)

Row(id='nc75237097', longitude=-122.817337036133, latitude=38.8286666870117, elevation=1.79999995231628, title='M 0.9 - 8 km NNW of The Geysers, CA', place_description='8 km NNW of The Geysers, CA', sig=12, mag=0.9, magType='md', time=datetime.datetime(2025, 9, 12, 23, 59, 57, 550000), updated=datetime.datetime(2025, 9, 13, 0, 27, 17, 978000))

In [26]:
# Save the transformed DataFrame to the Silver container
silver_data = f"{silver_adls}earthquake_events_silver/"

# append DataFrame to Silver container in Parquet format / (overwrite to test)
df.write.mode('overwrite').parquet(silver_data)

StatementMeta(Synapsedemo, 1, 8, Finished, Available, Finished)

In [ ]:
# passing metadata/output values to the next notebook
mssparkutils.notebook.exit(silver_data)